In [ ]:
Function to create a FASTAPI, create function for GET and POST and provide functionality to the application

In [1]:
from fastapi import FastAPI                                        #Importing FASTAPI for creation of API 
from pydantic import BaseModel                                     #Importing PYDANTIC BASEMODEL to accept user input in the URL 
from loguru import logger                                          #Importing LOGGER to check for logs and Debugging 

app = FastAPI()                                                    #Creation of FASTAPI

class UserRequestIn(BaseModel):                                    #Creation of User Input
    text: str

#Function for Testing        
        
@app.post("/test")                                                 
def index(request: UserRequestIn):
    logger.debug(request)
    return {"ok": True}
  
#GET Function for dictionary.txt to be returned as a JSON Object
    
@app.get("/dictionary")                                            
def dictionary_get():

    import pandas as pd                                            
    p ='dictionary.txt'
    dic = pd.read_fwf(p)
    return dic

# POST Function for dictionary.txt to accept keyword from the user and then add it to the file.

@app.post("/dictionary")                                           
def dictionary_post(request: UserRequestIn):
    
    with open("dictionary.txt", "a+") as f1:
        request=(str(request).split('='))
        f1.write('\n'+str(request[1]).replace("'",""))
        
    f1.close()

    '''This endpoint should be used to add words to the dictionary''' 
      
# POST Function for dictionary.txt to accept keyword from the user and then delete it from the file.
        
@app.delete("/dictionary")                                         
def dictionary_delete(request: UserRequestIn):
    
    f = open("dictionary.txt",'r')
    lines = f.readlines()
    f.close()

    request=(str(request).split('='))
    new_word = str(request[1]).replace("'","")

    newLines = []
    for line in lines:
        newLines.append(' '.join([word for word in line.split() if word != new_word]))

    f = open("dictionary.txt", 'w')
    for line in lines:
        f.write("{}\n".format(line))
    f.close()

# POST Function to accept contents of story.txt and search for keywords not in dictionary.txt. Additionally, return all the unmatched keywords along with their possible matches from dictionary.txt to accept keyword from the user and then add it to the file.    

@app.post("/story")
def story_post(request: UserRequestIn):                             

    'Using the posted body (story), the API should return a list of words that are not found or added to the dictionary of the system. For each word provide what the closest match for it would be from the dictionary. If more than one item is the same level of close match to a misspelled entity return the suggestion from the dictionary that comes first.'
    
    request=(str(request).split('='))
    string_input = str(request[1]).replace("'","")
    
    with open("dictionary.txt", "r") as f1:
        data=f1.readlines()
        newdata = []
        for word in data:
            newdata.append(word.replace("\n",""))

        string_input = string_input.split(' ')
        unmatch = []
        for word in string_input:
            if word not in newdata:
                unmatch.append(word)
        
        def match_with_gaps(my_word, other_word):
            
            my_word = my_word.replace(' ','')
            for char in my_word:
                if char.isalpha():
                    if my_word.count(char) != other_word.count(char):
                        break
                    if len(my_word) == len(other_word):
                        return True
        ans_dict =[]
        for w1 in newdata:
            for word in unmatch:
                check = match_with_gaps(word,w1)
                if check:
                    ans_dict.append({"word": word,
                           "closeMatch": w1})
        return ans_dict

Function to wrap ngrok and and make it available for Python API and create object that has a reference to the public URL.

In [2]:
from pyngrok import ngrok

ngrok_tunnel = ngrok.connect(8000)

ngrok_tunnel

<NgrokTunnel: "http://7066-153-33-85-163.ngrok.io" -> "http://localhost:8000">

Function to create a loop and keep the server running.

In [3]:
import nest_asyncio
import uvicorn

nest_asyncio.apply()
uvicorn.run(app, port=8000)

INFO:     Started server process [18212]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:56773 - "GET / HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:56773 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:56773 - "GET /openapi.json HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [18212]
